# 텍스트마이닝 데이터수집 프로그램

### [0] 사전 환경설정 및 함수정의
---

In [5]:
# 가상의 브라우저를 컨트롤 할 수 있도록 도와주는 selenium 패키지를 설치합니다.
# 아래 주석을 해지하고 셀을 실행합니다.
# 설치는 한번만 수행하면 되며, 재설치시 Requirement already satisfied: ~ 라는 메시지가 출력됩니다.

!pip3 install selenium

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip3 install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-23.2-py3-none-any.whl (2.1 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
conda install selenium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 23.5.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    selenium-3.141.0           |py38h1de35cc_1001         812 KB
    ------------------------------------------------------------
                                           Total:         812 KB

The following NEW packages will be INSTALLED:

  selenium           pkgs/main/osx-64::selenium-3.141.0-py38h1de35cc_1001



selenium-3.141.0     | 812 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated pac

In [1]:
# 기타 라이브러리
import time
import datetime as dt
import urllib.parse
import math

In [2]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

Chrome Driver 다운로드 URL : http://chromedriver.chromium.org/downloads

### 함수 1. 가상의 브라우저 실행하기: Chrome Driver 및 대상페이지 URL로 이동

---

In [3]:
# 함수 Parameter설명 
# URL : 크롬드라이버 드라이버 생성후 최초 접속할 URL
# Retrun 값은 없음

def exec_chrome_driver(URL):
    # Chrome Driver를 호출합니다.
    chrome_options = webdriver.ChromeOptions()

    # 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
    chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')

    # Chrome Driver 파일의 경로를 지정하고 실행합니다.
    # Chrome Driver는 아래 링크에서 다운로드 가능합니다.

    # 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

    # Windows 운영체제
    # driver = webdriver.Chrome(executable_path = "chromedriver", chrome_options=chrome_options)

    # MAC, Linux 운영체제
    driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)

    #URL로 리뷰페이지로 이동
    driver.get(URL)      
    time.sleep(2)
    
    #드라이버 변수 리턴
    return driver

### 함수 3. 금융데이터거래소 데이터 수집

---

In [7]:
# 함수 Parameter설명 
# driver : 크롬드라이버 드라이버 개체
# URL_before : 전체 URL에서 페이지번호를 제외한 앞부분
# URL_after : 전체 URL에서 페이지번호를 제외한 뒷부분
# max_pageno : 총 수집할 페이지수
# file_name : 수집 데이터 저장될 아웃풋 파일
# Retrun 값은 없음

def findatamall_getdata(driver, URL_before, URL_after, max_pageno, file_name):
    # 수집할 카페 게시물의 링크주소를 저장할 리스트를 생성합니다.
    post_list = []
    count = 0

    # 몇 페이지 까지 게시물의 URL을 수집할지 지정합니다.
    # 최대 페이지 수를 넘지 않도록 주의합니다.
    page_limit = max_pageno
    
    #데이터 수집 및 파일로 저장
    f = open(file_name, "w", encoding = "utf-8")

    # FOR 문을 활용해 페이지 번호를 반복합니다.
    for page_num in range(1, page_limit+1):
        # 검색결과 페이지로 이동합니다.
        driver.get(URL_before + str(page_num) + URL_after)
        
        time.sleep(2)
        
        # 회사명 가져오기
        box_list = driver.find_elements(By.CLASS_NAME, "data-listview-info")
            
        for box in box_list:          
            # 게시일자 가져오기
            tmp = box.find_element(By.CLASS_NAME,"meta-wrap.mobile-date")
            date_em = tmp.find_elements(By.TAG_NAME, "span")[0]
            date = date_em.text
            
            # 회사명 가져오기
            company_em = tmp.find_elements(By.TAG_NAME, "span")[1]
            company = company_em.text

            # 타이틀 가져오기
            title_em = box.find_element(By.CLASS_NAME, "title")
            title = title_em.text
            #title = box.find_element_by_css_selector("title break_word2")
    
            # 요약 가져오기
            summary_em = box.find_element(By.CLASS_NAME, "summary")
            summary= summary_em.text
            
            # 관심상품 수
            fav = box.find_element(By.CLASS_NAME,"total-fav")
            fav_em = fav.find_elements(By.TAG_NAME, "span")[1]
            favorite = fav_em.text

            # 평균별점
            rat = box.find_element(By.CLASS_NAME,"avg-rating")
            rat_em = rat.find_elements(By.TAG_NAME, "span")[1]
            rating = rat_em.text
            
            # 누적거래량
            dwld = box.find_element(By.CLASS_NAME,"total-download")
            dwld_em = dwld.find_elements(By.TAG_NAME, "span")[1]
            download = dwld_em.text
            
            # 누적조회 수
            vi = box.find_element(By.CLASS_NAME,"total-viewing")
            vi_em = vi.find_elements(By.TAG_NAME, "span")[1]
            view = vi_em.text
            
            # 가격
            pri = driver.find_element(By.CLASS_NAME,"data-listview-action")
            pri_em = pri.find_elements(By.TAG_NAME, "span")[0]
            price = pri_em.text
            
            #print(company.text + "\t" + title + "\t" + price.text + "\n")
            f.write(
                    date + "\t" 
                    + company + "\t" 
                    + title + "\t"
                    + summary + "\t" 
                    + favorite + "\t" 
                    + rating + "\t"
                    + download + "\t"
                    + view + "\t"
                    + price + "\t" 
                    + "\n")
            count+=1
            print(page_num,"/", page_limit," 페이지",", ", "총", count, "개 데이터 수집 중", end="\r")
        
        #for company, title, price in zip(company_list, title_list, price_list):
        #    print("for문 진입")
        #    print(company.text + "\t" + title.text + "\t" + price.text + "\n")
        #    f.write(company.text + "\t" + title.text + "\t" + price.text + "\n")
        #    count+=1
        #    print(page_num,"/", page_limit," 페이지",", ", "총", count, "개 데이터 수집 중", end="\r")
            
        time.sleep(2)

    print("")
    # 총 몇개의 게시물이 저장되었는지 확인
    print("총 ",page_num," 페이지",", ", "총", count, "개 데이터 수집 완료")
        
    f.close()
    
    # 파일에 저장된 카페 게시글 내용을 확인합니다.
    f = open(file_name, encoding="utf-8")
    for post in f:
        print(post.strip())
    f.close()

### [2] 금융데이터거래소 데이터 수집

---

In [8]:
driver = exec_chrome_driver("https://www.findatamall.or.kr/")

<ipython-input-3-4975daacae43>:21: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)


In [9]:
#금융데이터거래소 데이터 수집
findatamall_getdata(driver, 
            "https://www.findatamall.or.kr/fsec/dataProd/generalDataProd.do?cmnx=44&sCharge=&sFree=&sNego=&searchType=&sKeyword=&sTaxonomy=&sOrderByType=&sAvailableBuy=&page=", 
            "",
            1,
            "findx_test.txt")

1 / 1  페이지 ,  총 10 개 데이터 수집 중
총  1  페이지 ,  총 10 개 데이터 수집 완료
2023-07-20	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_강원_2021년 6월	2021년 6월 강원 지역 개인/기업별 카드 이용현황	0	0.0	0	8	무료
2023-07-20	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_경북_2021년 6월	2021년 6월 경북 지역 개인/기업별 카드 이용현황	0	0.0	0	3	무료
2023-07-20	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_경남_2021년 6월	2021년 6월 경남 지역 개인/기업별 카드 이용현황	0	0.0	0	2	무료
2023-07-20	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_경기_2021년 6월	2021년 6월 경기 지역 개인/기업별 카드 이용현황	0	0.0	0	3	무료
2023-07-14	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_2022년 2월	2022년 2월 개인/기업별 카드 이용현황	1	0.0	8	66	무료
2023-07-14	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_2022년 1월	2022년 1월 개인/기업별 카드 이용현황	0	0.0	1	17	무료
2023-07-14	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_2021년 12월	2021년 12월 개인/기업별 카드 이용현황	0	0.0	0	13	무료
2023-07-14	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_2021년 11월	2021년 11월 개인/기업별 카드 이용현황	0	0.0	0	15	무료
2023-07-14	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_2021년 10월	2021년 10월 개인/기업별 카드 이용현황	0	0.0	0	10	무료
2023-07-14	농협은행(농협카드)	[NH농협카드] 일자별 소비현황_2021년 9월	2021년 9월 개인/기업별 카드 이용현황	0	0.0	0	9	무료


In [31]:
#금융데이터거래소 데이터 수집
findatamall_getdata(driver, 
            "https://www.findatamall.or.kr/fsec/dataProd/generalDataProd.do?cmnx=44&sCharge=&sFree=&sNego=&searchType=&sKeyword=&sTaxonomy=&sOrderByType=&sAvailableBuy=&page=", 
            "",
            128,
            "findatamall_data_20220622.txt")

128 / 128  페이지 ,  총 1280 개 데이터 수집 중
총  128  페이지 ,  총 1280 개 데이터 수집 완료
2023-07-20	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_강원_2021년 6월	2021년 6월 강원 지역 개인/기업별 카드 이용현황	♡	4
2023-07-20	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_경북_2021년 6월	2021년 6월 경북 지역 개인/기업별 카드 이용현황	♡	1
2023-07-20	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_경남_2021년 6월	2021년 6월 경남 지역 개인/기업별 카드 이용현황	♡	1
2023-07-20	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_경기_2021년 6월	2021년 6월 경기 지역 개인/기업별 카드 이용현황	♡	2
2023-07-14	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_2022년 2월	2022년 2월 개인/기업별 카드 이용현황	♡	64
2023-07-14	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_2022년 1월	2022년 1월 개인/기업별 카드 이용현황	♡	17
2023-07-14	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_2021년 12월	2021년 12월 개인/기업별 카드 이용현황	♡	13
2023-07-14	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_2021년 11월	2021년 11월 개인/기업별 카드 이용현황	♡	15
2023-07-14	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_2021년 10월	2021년 10월 개인/기업별 카드 이용현황	♡	10
2023-07-14	농협은행(농협카드)	금융데이터거래소	[NH농협카드] 일자별 소비현황_2021년 9월	2021년 9월 개인/기업별 카드 이용현황	♡	9
2023-07-14	농협은행(농협카드)	금

In [32]:
driver.close()

In [33]:
import pandas as pd

In [47]:
df = pd.DataFrame(pd.read_csv('findatamall_data_20220622.txt',sep='\t'))

In [48]:
df

,2023-07-20,농협은행(농협카드),금융데이터거래소,[NH농협카드] 일자별 소비현황_강원_2021년 6월,2021년 6월 강원 지역 개인/기업별 카드 이용현황,♡,4
0,2023-07-20,농협은행(농협카드),금융데이터거래소,[NH농협카드] 일자별 소비현황_경북_2021년 6월,2021년 6월 경북 지역 개인/기업별 카드 이용현황,♡,1
1,2023-07-20,농협은행(농협카드),금융데이터거래소,[NH농협카드] 일자별 소비현황_경남_2021년 6월,2021년 6월 경남 지역 개인/기업별 카드 이용현황,♡,1
2,2023-07-20,농협은행(농협카드),금융데이터거래소,[NH농협카드] 일자별 소비현황_경기_2021년 6월,2021년 6월 경기 지역 개인/기업별 카드 이용현황,♡,2
3,2023-07-14,농협은행(농협카드),금융데이터거래소,[NH농협카드] 일자별 소비현황_2022년 2월,2022년 2월 개인/기업별 카드 이용현황,♡,64
4,2023-07-14,농협은행(농협카드),금융데이터거래소,[NH농협카드] 일자별 소비현황_2022년 1월,2022년 1월 개인/기업별 카드 이용현황,♡,17
...,...,...,...,...,...,...,...
1274,2023-06-12,하나카드,금융데이터거래소,"시군구별 업종별 가맹점 데이터(2022년 7월, 부산 기장군)",시군구별 업종별 가맹점 데이터,♡,15
1275,2023-06-12,하나카드,금융데이터거래소,"시군구별 업종별 가맹점 데이터(2022년 7월, 부산 금정구)",시군구별 업종별 가맹점 데이터,♡,4
1276,2023-06-12,하나카드,금융데이터거래소,"시군구별 업종별 가맹점 데이터(2022년 7월, 부산 강서구)",시군구별 업종별 가맹점 데이터,♡,15
1277,2023-06-12,하나카드,금융데이터거래소,"시군구별 업종별 가맹점 데이터(2022년 7월, 대전 중구)",시군구별 업종별 가맹점 데이터,♡,12


In [49]:
df.to_excel(
    excel_writer = 'findx.xlsx',
    sheet_name = 'Sheet1',
    index = False,       # 0부터 시작하는 자연수 인덱스는 의미가 없음.
    # columns = ['col1, 'col2', 'col3'],
    encoding = 'utf-8',
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = ''     # 무한값을 ''으로 채우기
)     # 해당 파일이 열려있으면 안됨.